# Multiple Linear Regression

In [24]:
import csv
import pandas as pd
import numpy as np
import sklearn
from sklearn import *
from sklearn.metrics import mean_squared_error,mean_absolute_error
from math import sqrt
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.feature_selection import RFE

/Users/emily-air13/anaconda3/lib/python3.6/site-packages/sklearn/cross_validation.py:41: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. Also note that the interface of the new CV iterators are different from that of this module. This module will be removed in 0.20.
  "This module will be removed in 0.20.", DeprecationWarning)
/Users/emily-air13/anaconda3/lib/python3.6/site-packages/sklearn/grid_search.py:42: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. This module will be removed in 0.20.
  DeprecationWarning)
/Users/emily-air13/anaconda3/lib/python3.6/site-packages/sklearn/learning_curve.py:22: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the functions are moved. This module will 

In [61]:
pd.set_option('display.max_columns',65)

In [62]:
df=pd.read_csv("../../clean_data.csv")

In [63]:
df.head()

,parcelid,airconditioningtypeid,bedroomcnt,buildingqualitytypeid,calculatedbathnbr,calculatedfinishedsquarefeet,fips,fireplacecnt,fullbathcnt,garagecarcnt,garagetotalsqft,hashottuborspa,heatingorsystemtypeid,latitude,longitude,lotsizesquarefeet,poolcnt,propertycountylandusecode,propertylandusetypeid,rawcensustractandblock,regionidcity,roomcnt,unitcnt,yearbuilt,numberofstories,fireplaceflag,structuretaxvaluedollarcnt,taxvaluedollarcnt,assessmentyear,landtaxvaluedollarcnt,taxamount,taxdelinquencyflag,taxdelinquencyyear,logerror,transactiondate,setyear,pooltype,zipcode
0,17073783,5.0,3.0,6.015461,2.5,1264.0,6111.0,0.0,2.0,2.0,434.0,False,2.0,34.303597,-119.287236,1735.0,0.0,1128,265.0,61110022003007,34543.0,5.0,1.0,1986.0,2.0,False,115087.0,191811.0,2015.0,76724.0,2015.06,N,NaN,0.0953,2016-01-27,2016,0,93001
1,17088994,5.0,2.0,6.015461,1.0,777.0,6111.0,0.0,1.0,1.0,434.0,False,2.0,34.272866,-119.198911,7200.0,0.0,1129,266.0,61110015031002,34543.0,4.0,1.0,1990.0,1.0,False,143809.0,239679.0,2015.0,95870.0,2581.30,N,NaN,0.0198,2016-03-30,2016,0,93004
2,17100444,5.0,3.0,6.015461,2.0,1101.0,6111.0,0.0,2.0,2.0,441.0,False,2.0,34.340801,-119.079610,6569.0,0.0,1111,261.0,61110007011007,26965.0,5.0,1.0,1956.0,1.0,False,33619.0,47853.0,2015.0,14234.0,591.64,N,NaN,0.0060,2016-05-27,2016,0,93066
3,17102429,5.0,2.0,6.015461,1.5,1554.0,6111.0,1.0,1.0,2.0,460.0,False,2.0,34.354313,-119.076405,7400.0,0.0,1110,261.0,61110008002013,26965.0,5.0,1.0,1965.0,1.0,True,45609.0,62914.0,2015.0,17305.0,682.78,N,NaN,-0.0566,2016-06-07,2016,0,93060
4,17109604,5.0,4.0,6.015461,2.5,2415.0,6111.0,1.0,2.0,2.0,665.0,False,2.0,34.266578,-119.165392,6326.0,0.0,1111,261.0,61110014021007,34543.0,8.0,1.0,1984.0,2.0,True,277000.0,554000.0,2015.0,277000.0,5886.92,N,NaN,0.0573,2016-08-08,2016,0,93004


In [64]:
def hottub(a):
    if a:
        return 1
    else:
        return 0
    
df['hashottuborspa'] = df['hashottuborspa'].map(lambda a: hottub(a))

def fire(a):
    if a:
        return 1
    else:
        return 0

df['fireplaceflag'] = df['fireplaceflag'].map(lambda a: fire(a))

def tax(a):
    if a=='Y':
        return 1
    else:
        return 0

df['taxdelinquencyflag'] = df['taxdelinquencyflag'].map(lambda a: tax(a))



fips = pd.get_dummies(df['fips'])
df = df.join(fips)

yr = pd.get_dummies(df['setyear'])
df = df.join(yr)

ac = pd.get_dummies(df['airconditioningtypeid'], prefix='ac')
df = df.join(ac)

heat = pd.get_dummies(df['heatingorsystemtypeid'], prefix='heat')
df=df.join(heat)

pool = pd.get_dummies(df['pooltype'], prefix='pooltype')
df = df.join(pool)

df.head()


,parcelid,airconditioningtypeid,bedroomcnt,buildingqualitytypeid,calculatedbathnbr,calculatedfinishedsquarefeet,fips,fireplacecnt,fullbathcnt,garagecarcnt,garagetotalsqft,hashottuborspa,heatingorsystemtypeid,latitude,longitude,lotsizesquarefeet,poolcnt,propertycountylandusecode,propertylandusetypeid,rawcensustractandblock,regionidcity,roomcnt,unitcnt,yearbuilt,numberofstories,fireplaceflag,structuretaxvaluedollarcnt,taxvaluedollarcnt,assessmentyear,landtaxvaluedollarcnt,taxamount,taxdelinquencyflag,taxdelinquencyyear,logerror,transactiondate,setyear,pooltype,zipcode,6037.0,6059.0,6111.0,2016,2017,ac_1.0,ac_3.0,ac_5.0,ac_9.0,ac_11.0,ac_13.0,heat_1.0,heat_2.0,heat_6.0,heat_7.0,heat_10.0,heat_11.0,heat_12.0,heat_13.0,heat_14.0,heat_18.0,heat_20.0,heat_24.0,pooltype_0,pooltype_2,pooltype_7,pooltype_10
0,17073783,5.0,3.0,6.015461,2.5,1264.0,6111.0,0.0,2.0,2.0,434.0,0,2.0,34.303597,-119.287236,1735.0,0.0,1128,265.0,61110022003007,34543.0,5.0,1.0,1986.0,2.0,0,115087.0,191811.0,2015.0,76724.0,2015.06,0,NaN,0.0953,2016-01-27,2016,0,93001,0,0,1,1,0,0,0,1,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,1,0,0,0
1,17088994,5.0,2.0,6.015461,1.0,777.0,6111.0,0.0,1.0,1.0,434.0,0,2.0,34.272866,-119.198911,7200.0,0.0,1129,266.0,61110015031002,34543.0,4.0,1.0,1990.0,1.0,0,143809.0,239679.0,2015.0,95870.0,2581.30,0,NaN,0.0198,2016-03-30,2016,0,93004,0,0,1,1,0,0,0,1,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,1,0,0,0
2,17100444,5.0,3.0,6.015461,2.0,1101.0,6111.0,0.0,2.0,2.0,441.0,0,2.0,34.340801,-119.079610,6569.0,0.0,1111,261.0,61110007011007,26965.0,5.0,1.0,1956.0,1.0,0,33619.0,47853.0,2015.0,14234.0,591.64,0,NaN,0.0060,2016-05-27,2016,0,93066,0,0,1,1,0,0,0,1,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,1,0,0,0
3,17102429,5.0,2.0,6.015461,1.5,1554.0,6111.0,1.0,1.0,2.0,460.0,0,2.0,34.354313,-119.076405,7400.0,0.0,1110,261.0,61110008002013,26965.0,5.0,1.0,1965.0,1.0,1,45609.0,62914.0,2015.0,17305.0,682.78,0,NaN,-0.0566,2016-06-07,2016,0,93060,0,0,1,1,0,0,0,1,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,1,0,0,0
4,17109604,5.0,4.0,6.015461,2.5,2415.0,6111.0,1.0,2.0,2.0,665.0,0,2.0,34.266578,-119.165392,6326.0,0.0,1111,261.0,61110014021007,34543.0,8.0,1.0,1984.0,2.0,1,277000.0,554000.0,2015.0,277000.0,5886.92,0,NaN,0.0573,2016-08-08,2016,0,93004,0,0,1,1,0,0,0,1,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,1,0,0,0


Variables in the Cleaned Data:

Location & Zoning Data (Potential Categorical X):
<div style="-webkit-column-count: 2; -moz-column-count: 2; column-count: 2; -webkit-column-rule: 1px dotted #e0e0e0; -moz-column-rule: 1px dotted #e0e0e0; column-rule: 1px dotted #e0e0e0;">
    <div style="display: inline-block;">
        <ul>
         <li>fips</li>
          <li>latitude</li>
        <li>longitude</li>
         <li>propertycountylandusecode</li>
         
         
         </ul>
    </div>
    <div style="display: inline-block;">
        <ul>
        <li>propertylandusetypeid</li>
         <li>rawcensustractandblock</li>
         <li>regionidcity</li>
         <li>zipcode</li>
        </ul>
    </div>
</div>

Categorical X:

<div style="-webkit-column-count: 2; -moz-column-count: 2; column-count: 2; -webkit-column-rule: 1px dotted #e0e0e0; -moz-column-rule: 1px dotted #e0e0e0; column-rule: 1px dotted #e0e0e0;">
    <div style="display: inline-block;">
        <ul>
         <li>airconditioningtypeid</li>
         <li>hashottuborspa</li>
         <li>heatingorsystemtypeid</li>
        <li>fireplaceflag</li>
        <li>assessmentyear</li>
        </ul>
    </div>
    <div style="display: inline-block;">
        <ul>
        <li>taxdelinquencyflag</li>
        <li>setyear</li>
        <li>pooltype</li> 
        <li>transactiondate</li>
        </ul>
    </div>
</div>


Numerical X:
<div style="-webkit-column-count: 2; -moz-column-count: 2; column-count: 2; -webkit-column-rule: 1px dotted #e0e0e0; -moz-column-rule: 1px dotted #e0e0e0; column-rule: 1px dotted #e0e0e0;">
    <div style="display: inline-block;">
        <ul>
        <li>bedroomcnt</li>
        <li>buildingqualitytypeid</li>
        <li>calculatedbathnbr</li>
        <li>calculatedfinishedsquarefeet</li>
        <li>fireplacecnt</li>
        <li>fullbathcnt</li>
        <li>garagecarcnt</li>
        <li>garagetotalsqft</li>
        <li>lotsizesquarefeet</li>
        <li>poolcnt</li>
        </ul>
    </div>
    <div style="display: inline-block;">
        <ul>
        <li>roomcnt</li>
        <li>unitcnt</li>
        <li>yearbuilt</li>
        <li>numberofstories</li>
        <li>structuretaxvaluedollarcnt</li>
        <li>taxvaluedollarcnt</li>
        <li>landtaxvaluedollarcnt</li>
        <li>taxamount</li>
        <li>taxdelinquencyyear TO REMOVE</li> 
        </ul>
    </div>
</div>
           
                        
Y:
* logerror

## Separate the Data Into Train and Test

In [65]:
array=df.values

In [67]:
Y=array[:,33]
Y

array([0.0953, 0.0198, 0.006, ..., 0.725458273984, 0.043652112878300016,
       -0.0141783454295], dtype=object)

In [68]:
x1=array[:,2:6]
x2=array[:,7:12]
x3=array[:,13:17]
x4=array[:,21:32]  
x5=array[:,38:65]

X=np.hstack((x1,x2))
X=np.hstack((X,x3))
X=np.hstack((X,x4))
X=np.hstack((X,x5))

X[1]

array([2.0, 6.015460983643267, 1.0, 777.0, 0.0, 1.0, 1.0, 434.0, 0,
       34.272866, -119.19891100000001, 7200.0, 0.0, 4.0, 1.0, 1990.0, 1.0,
       0, 143809.0, 239679.0, 2015.0, 95870.0, 2581.3, 0, 0, 0, 1, 1, 0, 0,
       0, 1, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0], dtype=object)

In [69]:
X_train,X_test,Y_train,Y_test=train_test_split(X,Y,test_size=0.2)

## Basic Linear Regression

In [70]:
lm=linear_model.LinearRegression()

lm.fit(X_train, Y_train)

LinearRegression(copy_X=True, fit_intercept=True, n_jobs=1, normalize=False)

In [71]:
lm.intercept_

-2.3384939707186572

In [72]:
lm.coef_

array([ -2.41332331e-04,  -1.35045619e-03,  -3.69931192e-03,
         1.22467474e-05,   1.76503013e-03,   2.96589010e-03,
         3.46983932e-03,  -2.61940652e-05,  -6.11428797e-03,
        -2.64628922e-03,   5.27066168e-03,   2.91923735e-08,
        -5.52544828e-03,   8.09439073e-04,  -7.96492310e-03,
         9.16023661e-06,   8.43116785e-04,  -9.74039214e-03,
        -3.46891866e-07,   3.72460788e-07,   1.51481440e-03,
        -3.42786852e-07,  -3.00022461e-06,   2.00730658e-02,
        -6.38978948e-03,  -1.75036008e-03,   8.14014956e-03,
        -1.51481440e-03,   1.51481440e-03,  -1.88904563e-02,
         9.30682014e-02,  -1.70701216e-02,  -3.79752669e-02,
         2.84725119e-04,  -1.94170817e-02,   3.66716747e-02,
         1.22073172e-02,   1.26626083e-02,   1.36346766e-02,
        -3.03877815e-02,  -5.54299899e-02,  -4.88886208e-04,
        -2.94865456e-03,   3.57775182e-03,   2.33339585e-02,
        -4.20599369e-03,  -8.62668115e-03,   4.37934226e-03,
        -7.26039409e-03,

In [73]:
ptrain = lm.predict(X_train)
ptest = lm.predict(X_test)

Compare the goodness of fit of the model for the train set versus the test set

In [75]:
sqrt(mean_squared_error(Y_train, ptrain))

0.16360940222540835

In [31]:
sqrt(mean_squared_error(Y_test, ptest))

0.17031177521389948

In [76]:
mean_absolute_error(Y_train, ptrain)

0.069055566822086981

In [77]:
mean_absolute_error(Y_test, ptest)

0.070423539971245716

## Lasso Feature Selection

In [78]:
clf = linear_model.Lasso(tol=1) #alpha=0.1
clf.fit(X_train, Y_train)

Lasso(alpha=1.0, copy_X=True, fit_intercept=True, max_iter=1000,
   normalize=False, positive=False, precompute=False, random_state=None,
   selection='cyclic', tol=1, warm_start=False)

In [79]:
clf.intercept_

0.0032004941593023126

In [80]:
clf.coef_

array([  0.00000000e+00,  -0.00000000e+00,   0.00000000e+00,
         7.57157516e-06,  -0.00000000e+00,   0.00000000e+00,
         0.00000000e+00,   0.00000000e+00,  -0.00000000e+00,
        -0.00000000e+00,   0.00000000e+00,   1.71259738e-08,
        -0.00000000e+00,   0.00000000e+00,  -0.00000000e+00,
         0.00000000e+00,   0.00000000e+00,  -0.00000000e+00,
        -3.61807239e-09,  -1.49698166e-09,   0.00000000e+00,
        -1.00452889e-09,  -2.29103881e-07,   0.00000000e+00,
        -0.00000000e+00,   0.00000000e+00,   0.00000000e+00,
        -0.00000000e+00,   0.00000000e+00,  -0.00000000e+00,
         0.00000000e+00,   0.00000000e+00,  -0.00000000e+00,
         0.00000000e+00,   0.00000000e+00,   0.00000000e+00,
         0.00000000e+00,  -0.00000000e+00,   0.00000000e+00,
        -0.00000000e+00,  -0.00000000e+00,  -0.00000000e+00,
        -0.00000000e+00,  -0.00000000e+00,   0.00000000e+00,
        -0.00000000e+00,  -0.00000000e+00,   0.00000000e+00,
        -0.00000000e+00,

In [81]:
ptrain = clf.predict(X_train)
ptest = clf.predict(X_test)

Compare the goodness of fit of the model for the train set versus the test set

In [82]:
sqrt(mean_squared_error(Y_train, ptrain))

0.1638646429667453

In [83]:
sqrt(mean_squared_error(Y_test, ptest))

0.17111287742993772

In [84]:
mean_absolute_error(Y_train, ptrain)

0.06895492080469999

In [85]:
mean_absolute_error(Y_test, ptest)

0.070343658681989857

##  Recursive Feature Elimination

In [86]:
#http://blog.datadive.net/selecting-good-features-part-iv-stability-selection-rfe-and-everything-side-by-side/
model=LinearRegression()
rfe = RFE(model, n_features_to_select=25)
rfe.fit(X_train,Y_train)

RFE(estimator=LinearRegression(copy_X=True, fit_intercept=True, n_jobs=1, normalize=False),
  n_features_to_select=25, step=1, verbose=0)

In [87]:
rfe.ranking_ 

array([17, 16, 18, 22,  9, 14, 19, 20,  1, 11, 10, 27,  5, 13, 12, 21, 15,
        8, 25, 24,  6, 26, 23,  7,  1,  1,  1,  1,  1,  2,  1,  3,  4,  1,
        1,  1,  1,  1,  1,  1,  1,  1,  1,  1,  1,  1,  1,  1,  1,  1,  1])

In [88]:
ptrain=rfe.predict(X_train)
ptest=rfe.predict(X_test)

Compare the goodness of fit of the model for the train set versus the test set

In [89]:
sqrt(mean_squared_error(Y_train, ptrain))

0.16396855571413285

In [90]:
sqrt(mean_squared_error(Y_test, ptest))

0.17132238396104527

In [91]:
mean_absolute_error(Y_train, ptrain)

0.069239180763875433

In [92]:
mean_absolute_error(Y_test, ptest)

0.070646706395024725

## Backward Selection

We are backward selecting features using the MAE for the test data

In [94]:
train,test=train_test_split(array,test_size=0.2)

In [153]:
blm=linear_model.LinearRegression()
blm.fit(train[:,[5,7,13,21,25,26,29,31,38,39,40,43,45,47,53,57,61,62,63,64]], train[:,33])
ptest=blm.predict(test[:,[5,7,13,21,25,26,29,31,38,39,40,43,45,47,53,57,61,62,63,64]])
mean_absolute_error(test[:,33], ptest)

0.069596211914267986

For the sake of readablility we aren't showing each individual delete and add, but here are the final results:

Keep: 64,63,62,61,57,53,47,45,43,40,39,38,31,29,26,25,21,13,7,5

Delete: 60,59,58,56,55,54,52,51,50,49,48,46,44,42,41,30,27,24,22,16,15,14,11,10,9,8,4,3,2

The "keep" columns translate to:
* calculatedfinishedsquarefeet
* fireplacecnt
* latitude
* roomcnt
* fireplaceflag
* structuretaxvaluedollarcnt
* landtaxvaluedollarcnt
* taxdelinquencyflag
* fips codes
* A/C types 1, 5, 11
* heat types 10, 14
* pooltypes

Let's look at the statistics for this regression model:

In [156]:
ptrain= blm.predict(train[:,[5,7,13,21,25,26,29,31,38,39,40,43,45,47,53,57,61,62,63,64]])
sqrt(mean_squared_error(train[:,33], ptrain))

0.16575785143478808

In [157]:
sqrt(mean_squared_error(test[:,33], ptest))

0.16336582534409633

In [158]:
mean_absolute_error(train[:,33],ptrain)

0.06911780840679195

In [159]:
mean_absolute_error(test[:,33],ptest)

0.069596211914267986

## Forward Selection

Some of the improvements in MAE were very small, and the order in which columns were removed could have affected the results. Let's see if we can further improve the scores by forward selecting from the columns chosen by backward selection, adding them in the opposite order

In [184]:
flm=linear_model.LinearRegression()
flm.fit(train[:,[5,7,13,25,31,40,45,57,61,62]], train[:,33])
ptest=flm.predict(test[:,[5,7,13,25,31,40,45,57,61,62]])
mean_absolute_error(test[:,33], ptest)

0.069504850047729591

Remove: 21,26,29,38,39,43,47,53,63,64

The columns we are keeping are:
* calculatedfinishedsquarefeet
* fireplacecnt
* latitude
* fireplaceflag
* taxdelinquencyflag
* FIPS code 6111 (Ventura County)
* A/C type 5 (None)
* Heat type 14 ("Other")
* No pool or pool type 2 (pool with hot tub/spa)

Let's look at the statistics:

In [186]:
ptrain= flm.predict(train[:,[5,7,13,25,31,40,45,57,61,62]])
sqrt(mean_squared_error(train[:,33], ptrain))

0.16580236306254062

In [187]:
sqrt(mean_squared_error(test[:,33], ptest))

0.16339726635224958

In [188]:
mean_absolute_error(train[:,33], ptrain)

0.069027045083357205

In [189]:
mean_absolute_error(test[:,33], ptest)

0.069504850047729591